In [18]:
import os
import numpy as np
import keras
from keras import layers, models, callbacks
from keras.preprocessing.image import ImageDataGenerator

In [19]:
categories = [
    'aircraft carrier', 'airplane', 'ambulance', 'ant', 'anvil',
    'apple', 'axe', 'backpack', 'banana', 'baseball bat', 'baseball',
    'bat', 'bear', 'bed', 'bee', 'belt', 'bench', 'bird', 'book',
    'boomerang', 'bowtie', 'brain', 'bread', 'broom', 'bucket',
    'bulldozer', 'bus', 'The Eiffel Tower', 'The Mona Lisa'
]

data_path = './data/'

samples_per_category = 7000

X = []
y = []

for i, cat in enumerate(categories):
    file_path = os.path.join(data_path, f'full_numpy_bitmap_{cat}.npy')
    data = np.load(file_path)
    data = data[:samples_per_category]
    X.append(data)
    y.append(np.full(len(data), i))

X = np.concatenate(X).reshape(-1, 28, 28, 1).astype('float32') / 255.0
y = np.concatenate(y)

indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X, y = X[indices], y[indices]


print(f'X: {X.shape}, y: {y.shape}')

X: (203000, 28, 28, 1), y: (203000,)


In [20]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

In [21]:
model = models.Sequential([
    layers.Input(shape=(28, 28, 1)),

    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPool2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPool2D((2, 2)),

    layers.GlobalAveragePooling2D(),

    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(len(categories), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:

train_gen = datagen.flow(X,y, batch_size=128, subset='training')
val_gen = datagen.flow(X,y,batch_size=128, subset='validation')

lr_reducer = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
early_stopper = callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    validation_split=0.2,
    epochs=30,
    batch_size=128,
    callbacks=[lr_reducer, early_stopper]
)

model.save('draw-ai/models/temp_model.keras')

Epoch 1/30
1269/1269 [==============================] - 82s 64ms/step - loss: 1.3203 - accuracy: 0.6251 - val_loss: 0.8455 - val_accuracy: 0.7634 - lr: 0.0010
Epoch 2/30
1269/1269 [==============================] - 113s 89ms/step - loss: 0.8488 - accuracy: 0.7611 - val_loss: 0.7617 - val_accuracy: 0.7841 - lr: 0.0010
Epoch 3/30
1269/1269 [==============================] - 117s 92ms/step - loss: 0.7589 - accuracy: 0.7880 - val_loss: 0.6818 - val_accuracy: 0.8100 - lr: 0.0010
Epoch 4/30
1269/1269 [==============================] - 124s 98ms/step - loss: 0.7102 - accuracy: 0.8016 - val_loss: 0.6942 - val_accuracy: 0.8028 - lr: 0.0010
Epoch 5/30
1269/1269 [==============================] - 131s 103ms/step - loss: 0.6804 - accuracy: 0.8114 - val_loss: 0.6326 - val_accuracy: 0.8222 - lr: 0.0010
Epoch 6/30
1269/1269 [==============================] - 117s 92ms/step - loss: 0.6536 - accuracy: 0.8185 - val_loss: 0.6520 - val_accuracy: 0.8152 - lr: 0.0010
Epoch 7/30
1269/1269 [==================